In [1]:
from AREgym import AREEnv

import gym


from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env

from stable_baselines3.ppo import MlpPolicy

from torch.utils.tensorboard import SummaryWriter
# env = env = gym.make('Pendulum-v0')
# env = AREEnv(250,10)
import numpy as np

# env = AREEnv(250, 10, num_laser_scan=10, heuristic_dist=10, save_map=True)
# # print(env.reset())
# # env.render()

# for i in range(2000):
#     env.step((np.random.rand() - 0.5) * 2)
# # env.laserscan
# print(env.step(1))
# env.render()



In [2]:
env = AREEnv(250, 10, num_laser_scan=10, save_map=True, max_steps=256)
# env = AREEnv(500, 20, save_map=True)
# check_env(env)


In [3]:
# print(env.reset())


In [4]:
# policy_kwargs = dict(net_arch=dict(pi=[256, 128, 64], vf=[256, 128, 64]))
# import torch as th
policy_kwargs = dict(net_arch=[dict(pi=[128, 64], vf=[128, 64])])

In [6]:
model = PPO("MlpPolicy",
            env,
            verbose=0,
            n_steps=256,
            tensorboard_log='sb_runs/256',
            batch_size=256,
            # policy_kwargs=policy_kwargs,
            seed=1
            )
# model = PPO("MlpPolicy", env, verbose=1, gamma=0.9)
model.learn(total_timesteps=500000)
model.save("stable_baselines3_ppo")

In [17]:
model = PPO.load("stable_baselines3_ppo")

In [7]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (shared_net): Sequential()
    (policy_net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=1, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [9]:
obs = env.reset()

for i in range(256):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    print(action)
    # env.render()
env.render()

[0.85449046]
[-0.17770231]
[0.0512932]
[0.01544151]
[0.8458177]
[0.26850048]
[0.80516356]
[0.00955507]
[-0.16068348]
[0.83325016]
[-0.16517965]
[0.9527669]
[-0.17193802]
[-0.22006276]
[0.6592373]
[0.56241417]
[1.]
[-0.17402622]
[0.764632]
[-0.0273408]
[0.53615576]
[-0.04616368]
[1.]
[0.68531215]
[0.33948737]
[0.3157801]
[0.32607362]
[0.848341]
[0.31224686]
[1.]
[0.5039716]
[0.64345044]
[0.98298776]
[0.59078705]
[-0.40609986]
[0.690559]
[-0.09167241]
[0.36130643]
[-0.48306203]
[0.09427592]
[0.12064669]
[0.2629965]
[1.]
[-0.2945785]
[0.7480025]
[-0.6088873]
[0.51231974]
[1.]
[0.02365316]
[0.7257173]
[-1.]
[0.34264445]
[0.05042593]
[0.18008253]
[-0.18898925]
[1.]
[1.]
[1.]
[0.13784462]
[-0.8617697]
[-0.54209405]
[-0.60984564]
[-0.7393507]
[-0.8670524]
[-0.529429]
[-1.]
[-0.357852]
[-0.46068895]
[-0.42973053]
[-0.18329543]
[-0.5163065]
[-0.75061405]
[-0.4502393]
[-0.671586]
[-1.]
[-1.]
[-0.5924535]
[-0.828632]
[-0.90027356]
[-0.27692652]
[-0.7566738]
[-0.5432587]
[-0.58424795]
[-0.90541077

In [12]:
print(env.reset())
env.render()

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.64840183 0.76712329
 0.76940639 0.7716895  0.78538813 0.84018265 0.9109589  0.79452055
 1.         0.98858447]


In [10]:
print(obs)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
# import gym

# from stable_baselines3 import PPO
# from stable_baselines3.common.env_util import make_vec_env

# # Parallel environments
# env = make_vec_env("CartPole-v1", n_envs=4)

# model = PPO("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=25000)
# model.save("ppo_cartpole")

# del model # remove to demonstrate saving and loading

# model = PPO.load("ppo_cartpole")

# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     env.render()